In [1]:

import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
tabla_completa = pd.read_csv('events_up_to_01062018.csv');
tabla_completa.info()

C:\Users\Santi\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


In [3]:
res = pd.read_csv('labels_training_set.csv');
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Data columns (total 2 columns):
person    19414 non-null object
label     19414 non-null int64
dtypes: int64(1), object(1)
memory usage: 303.4+ KB


In [4]:
tabla_completa['aux']=1
#Voy a contar la cantidad para cada tipo de evento de cada persona
num_eventos = tabla_completa.groupby(['person','event'])['aux'].sum()

num_eventos = num_eventos.unstack().reset_index().fillna(0)
num_eventos = num_eventos.set_index('person')
num_eventos['total_eventos'] = num_eventos.sum(axis=1)
#normalizo los numeros obtenidos según la cantidad de eventos totales de esa persona
num_eventos_norm = num_eventos.loc[:,"ad campaign hit":"visited site"].div(num_eventos["total_eventos"], axis=0)
#num_eventos.sort_values(by='conversion',ascending=False)
#num_eventos_norm.sort_values(by='conversion',ascending=False)
num_eventos_norm = num_eventos_norm.reset_index()
num_eventos_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38829 entries, 0 to 38828
Data columns (total 12 columns):
person               38829 non-null object
ad campaign hit      38829 non-null float64
brand listing        38829 non-null float64
checkout             38829 non-null float64
conversion           38829 non-null float64
generic listing      38829 non-null float64
lead                 38829 non-null float64
search engine hit    38829 non-null float64
searched products    38829 non-null float64
staticpage           38829 non-null float64
viewed product       38829 non-null float64
visited site         38829 non-null float64
dtypes: float64(11), object(1)
memory usage: 3.6+ MB


In [10]:
tabla_completa['timestamp'] = pd.to_datetime(tabla_completa['timestamp'])
#tabla_completa['timestamp'].max() -tabla_completa['timestamp'].min()

time_for_user = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.max() - x.min())
time_for_user = time_for_user.reset_index()
time_for_user['total_horas'] = time_for_user['timestamp'].dt.total_seconds()/3600
time_for_user.drop(['timestamp'],axis='columns')

,person,timestamp,total_horas
0,0008ed71,0 days 04:00:50,4.013889
1,00091926,27 days 21:43:34,669.726111
2,00091a7a,0 days 00:05:47,0.096389
3,000ba417,9 days 01:57:37,217.960278
4,000c79fe,0 days 00:10:20,0.172222
5,000e4d9e,14 days 05:50:27,341.840833
6,000e619d,2 days 22:05:30,70.091667
7,001001be,5 days 18:07:28,138.124444
8,0010e89a,0 days 00:00:43,0.011944
9,0016c4b5,0 days 00:00:02,0.000556


In [ ]:
data_nuevo = pd.merge(num_eventos_norm,time_for_user,on='person',how='inner')

In [ ]:
intento = data_nuevo.drop(['conversion'],axis='columns')
intento_f = pd.merge(intento,res,on='person',how='inner')

intento_f = intento_f.set_index('person')
intento_f.info()

In [ ]:
final = pd.read_csv('trocafone_kaggle_test.csv')
final = pd.merge(intento,final,on='person',how='inner')
final = final.set_index('person')
final.info()

In [ ]:
x_entrega = final

In [ ]:
#XGBoost

X, y = intento_f.iloc[:,:-1],intento_f.iloc[:,-1]
X

In [ ]:
y

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,y_train)

In [ ]:
preds = xg_reg.predict(X_test)


In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
entrega = xg_reg.predict(x_entrega)
y_summit = pd.DataFrame(entrega)
y_summit.columns = ['label']
y_summit['person'] = x_entrega.reset_index()['person']
y_person = x_entrega.reset_index()['person']

y_summit = y_summit[['person','label']]
y_summit.to_csv('summit01.csv', index=False)

In [ ]:
#K-fold

params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="rmse", as_pandas=True, seed=123)

In [ ]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10}

In [ ]:
# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for alpha in range (0,20):
    # Update our parameters
    params['alpha'] = alpha
    # Run CV
    cv_results = xgb.cv(
        params=params,
        dtrain=data_dmatrix,
        num_boost_round=50,
        seed=123,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\trmse {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = alpha
print(best_params)